## Actual Implementation

In [16]:
# import statements
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [15]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


## initalisation

## Sector Dictionary
0. Category 1 (Agri)
1. Category 2 (Mining)
2. Category 3 (Construction)
3. Category 4 (Textile)
4. Category 5 (Transport Svcs)
5. Category 6 (ICT)
6. Category 7 (Health, pharm, sports etc)
7. Category 8 (Govt, Millitary, Misc)

In [ ]:
# temp=pd.read_csv('../data/final/final_training_model_data.csv',header=0)
# temp.head()
# temp.drop(columns=['Unnamed: 0'],inplace=True)
# temp=temp[(temp['country_a']!='ARE')& (temp['country_b']!='ARE')]
# temp.to_csv('../data/final/without_ARE.csv')

,Unnamed: 0,country_a,country_b,bec_1,bec_2,bec_3,bec_4,bec_5,bec_6,bec_7,bec_8,D,year
0,0,ARE,AUS,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.060739e+09,0.620151,2006
1,1,ARE,CHE,3.797882e+06,1.355991e+06,6.939408e+06,3.759398e+08,2.690339e+06,9.364289e+05,4.422947e+07,9.824469e+07,0.586050,2006
2,2,ARE,CHN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.953953e+09,0.635445,2006
3,3,ARE,DEU,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.159614e+08,0.566027,2006
4,4,ARE,FRA,4.972880e+07,4.680296e+07,9.306163e+07,7.307789e+07,9.781168e+07,1.751175e+07,1.162716e+08,2.462986e+08,0.620551,2006


In [19]:
num_countries=17
num_country_pairs=num_countries*(num_countries-1) 
num_sectors=8 # 8 sectors

class Args:
    def __init__(self):
        # Model structure
        self.num_nodes = num_country_pairs  
        self.input_dim = num_sectors+1    # e.g. sectorial export volume + composite
        self.rnn_units = 66
        self.output_dim = num_sectors   # e.g., predict only the sectorial export volume
        self.horizon = 3      # forecast 3 steps ahead
        self.num_layers = 1
        self.cheb_k = 2
        self.embed_dim = 6
        self.default_graph = True  
        self.log_dir = './logs/'
        self.debug = False
        self.model='AGCRN'
        self.normaliser = 'max11'
        self.device='cpu'
        self.batch_size=4 # 4/7 depending on results
        self.mode='train'
        # Training
        self.seed=10
        self.loss_func= 'mse'
        self.epochs = 50
        self.lr_init = 0.0059745480826864684
        self.lr_decay = True
        self.lr_decay_step = '5,20,40,70'
        self.lr_decay_rate = 0.17039700074755165
        self.early_stop = True
        self.early_stop_patience = 5
        self.teacher_forcing = True
        self.tf_decay_steps = 20
        self.real_value = False
        self.grad_norm = True
        self.max_grad_norm = 5
        self.weight_decay = 0.0001
        # Testing
        self.mae_thresh=None
        self.mape_thresh=0.

        #Logging
        self.log_step = 20
        self.plot=True

    def set_args(self, **kwargs):
        """
        Update attributes of Args dynamically based on the kwargs.
        If a key doesn't match an existing attribute, a warning is printed.
        """
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                print(f"Warning: '{key}' is not a recognized attribute of the Args class.")


args = Args()

## Data handling

In [ ]:
# temp=pd.read_csv('../data/final/without_ARE.csv',header=0) #i will only have 2007 to 2023 now
# temp_only_d=temp[['D','country_a','country_b','year']]
# temp.drop(columns=['Unnamed: 0','D'],inplace=True)
# temp.sort_values(by=['country_a','country_b','year'],inplace=True)
# bec_columns = [f'bec_{i}' for i in range(1, 9)]
# for col in bec_columns:
#     # Create a new column to store the percentage change.
#     temp[f'pct_{col}'] = temp.groupby(['country_a', 'country_b'])[col].pct_change() * 100
# temp.dropna(inplace=True)
# temp.reset_index(drop=True,inplace=True)
# temp=temp.merge(temp_only_d, on=['country_a','country_b','year'], how='left')
# temp.drop(columns=bec_columns,inplace=True)
# temp.rename(columns={'pct_bec_1':'bec_1','pct_bec_2':'bec_2','pct_bec_3':'bec_3','pct_bec_4':'bec_4','pct_bec_5':'bec_5','pct_bec_6':'bec_6','pct_bec_7':'bec_7','pct_bec_8':'bec_8'},inplace=True)
# temp.to_csv('../data/final/without_ARE_pct.csv',index=False)
# fbic_data=fbic_data.rename(columns={'iso3a':'country_a','iso3b':'country_b'})
# training_data=pd.read_csv('../data/final/training_model_data.csv',header=0)
# training_data=training_data[['country_a','country_b','bec_1','bec_2','bec_3','bec_4','bec_5','bec_6','bec_7','bec_8','sentiment_index','tradeagreementindex','year']]

In [ ]:
# temp=pd.read_csv('../data/final/compiled_model_data.csv',header=0)
# temp=temp.reset_index(drop=True)
# temp=temp[['country_a','country_b','bec_1','bec_2','bec_3','bec_4','bec_5','bec_6','bec_7','bec_8','D','year']]
# temp=temp[(temp['year']>=2006) & (temp['year']<=2020)]
# temp.to_csv('../data/final/final_training_model_data.csv')

# TRAINING

## Import data 

### Data transformation to pipeline data into model 

In [3]:
def csv_to_tensor(csv_file):
    """
    Reads a CSV file with columns:
      country1, country2, sector1, sector2, ..., sector8, sentiment, year
    and returns a tensor of shape (T, N, D), where:
      T = number of years,
      N = number of unique country pairs,
      D = num of sectors + features.
    Also returns the sorted list of years and country pair nodes.
    """
    # Read the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Ensure the 'year' column is integer (if needed)
    df['year'] = df['year'].astype(int)
    
    # Get a sorted list of unique years
    years = sorted(df['year'].unique())
    T = len(years)
    
    # Get all unique country pairs
    pairs_df = df[['country_a', 'country_b']].drop_duplicates()
    # Create a sorted list of tuples (country1, country2) for consistent node ordering
    country_pairs = sorted([tuple(x) for x in pairs_df.values])
    N = len(country_pairs)
    
    # Number of features (8 sectors + 1 sentiment)
    D = 9

    # Initialize an empty numpy array for the tensor data
    tensor_data = np.empty((T, N, D), dtype=float)
    
    # Loop over each year and each country pair to fill in the tensor
    for t, year in enumerate(years):
        # Get data for the current year
        df_year = df[df['year'] == year]
        for n, (c1, c2) in enumerate(country_pairs):
            # Filter rows for the current country pair
            row = df_year[(df_year['country_a'] == c1) & (df_year['country_b'] == c2)]
            if not row.empty:
                # Extract the 8 sector columns and the sentiment column.
                # Assumes these columns are named exactly as shown.
                features = row.iloc[0][['bec_1', 'bec_2', 'bec_3', 'bec_4', 
                                         'bec_5', 'bec_6', 'bec_7', 'bec_8', 'D']].values
                tensor_data[t, n, :] = features.astype(float)
            else:
                # If a record is missing for a given year/country pair, fill with zeros (or choose another strategy)
                tensor_data[t, n, :] = np.zeros(D)
                
    return tensor_data, years, country_pairs

def group_into_windows(tensor_data, window_size):
    """
    Given a tensor of shape (T, N, D), group the data into overlapping windows.
    Each window is of length window_size
    Returns a numpy array of shape (num_samples, window_size, N, D).
    """
    T, N, D = tensor_data.shape
    num_samples = T - window_size + 1  # sliding window with stride 1
    windows = []
    for i in range(num_samples):
        window = tensor_data[i: i + window_size]  # shape: (window_size, N, D)
        windows.append(window)
    windows = np.stack(windows)  # shape: (num_samples, window_size, N, D)
    return windows

def split_input_target_direct(windows, input_len, horizon=3):
    """
    Splits each window into input and a single target that is horizon steps forward.
    
    windows: numpy array of shape (num_samples, window_size, N, D)
              where window_size = input_len + horizon.
    input_len: number of time steps used as input.
    horizon: steps forward to pick the target (here, horizon=3).
    
    Returns:
      x: inputs of shape (num_samples, input_len, N, D)
      y: targets of shape (num_samples, N, 8), which are the first 8 features of the target time step.
    """
    # x: first input_len time steps (e.g., years 2006-2009 if input_len=4)
    x = windows[:, :input_len]  
    # y_full: the time step exactly horizon steps forward (i.e., index input_len + horizon - 1)
    # y_full = windows[:, input_len + horizon-1]  
    y_full = windows[:, input_len:input_len + horizon]
    # y: only the first 8 features from the predicted time step (ignoring sentiment_index and tradeagreementindex)
    y = y_full[..., :8]
    return x, y

def train_val_split(x, y, val_ratio=0.2):
    """
    Splits the data into train and validation sets by ratio.
    """
    num_samples = x.shape[0]
    split_index = int(num_samples * (1 - val_ratio))
    x_train, y_train = x[:split_index], y[:split_index]
    x_val, y_val = x[split_index:], y[split_index:]
    return x_train, y_train, x_val, y_val



In [ ]:
from AGCRN.lib.dataloader import normalize_dataset

#convert csv to tensor
training_data_tensor, years, country_pairs = csv_to_tensor('./data/final/without_ARE_pct.csv')

In [5]:
# Inspect data to check
print("Data for year {}:".format(years[0]))
print(training_data_tensor[0])  # prints the data for all nodes/features for the first year

print("Features for {} in {}:".format(country_pairs[0], years[0]))
print(training_data_tensor[0, 0, :]) # prints the features for the first country pair in the first year

Data for year 2007:
[[288.52271933 279.03809661 204.2020891  ... 174.11212633 196.4292726
    0.55853787]
 [ 38.17646947  31.0278394   36.47753454 ...  63.61518823 -17.67302399
    0.56009085]
 [ -0.29719578   3.56841464  14.62732324 ...  20.83388439 -32.08029114
    0.59298115]
 ...
 [ -0.33857947  18.18635501  26.43173842 ...  16.49221368  18.26733175
    0.3671412 ]
 [ -1.33240313  -4.83563772  47.63716963 ... -16.34540145  10.97735679
    0.46350549]
 [ 27.02208384 -18.73648827  26.17908911 ...  44.25435862  25.63200196
    0.57982181]]
Features for ('AUS', 'CHE') in 2007:
[2.88522719e+02 2.79038097e+02 2.04202089e+02 1.18079757e+03
 1.39229930e+02 6.28642389e+02 1.74112126e+02 1.96429273e+02
 5.58537867e-01]


## Country Dictionary (ordered by alphabetical order)

0. Australia (AUS)
1. Switzerland (CHE)
2. China (CHN)
3. Germany (DEU)
4. France (FRA)
5. Hong Kong, China (HKG)
6. Indonesia (IDN)
7. India (IND)
8. Japan (JPN)
9. Korea, Rep. (KOR)
10. Malaysia (MYS)
11. Netherlands (NLD)
12. Philippines (PHL)
13. Singapore (SGP)
14. Thailand (THA)
15. United States (USA)
16. Vietnam (VNM)

### List of country pairs for reference

In [ ]:
country_pairs

[('AUS', 'CHE'),
 ('AUS', 'CHN'),
 ('AUS', 'DEU'),
 ('AUS', 'FRA'),
 ('AUS', 'HKG'),
 ('AUS', 'IDN'),
 ('AUS', 'IND'),
 ('AUS', 'JPN'),
 ('AUS', 'KOR'),
 ('AUS', 'MYS'),
 ('AUS', 'NLD'),
 ('AUS', 'PHL'),
 ('AUS', 'SGP'),
 ('AUS', 'THA'),
 ('AUS', 'USA'),
 ('AUS', 'VNM'),
 ('CHE', 'AUS'),
 ('CHE', 'CHN'),
 ('CHE', 'DEU'),
 ('CHE', 'FRA'),
 ('CHE', 'HKG'),
 ('CHE', 'IDN'),
 ('CHE', 'IND'),
 ('CHE', 'JPN'),
 ('CHE', 'KOR'),
 ('CHE', 'MYS'),
 ('CHE', 'NLD'),
 ('CHE', 'PHL'),
 ('CHE', 'SGP'),
 ('CHE', 'THA'),
 ('CHE', 'USA'),
 ('CHE', 'VNM'),
 ('CHN', 'AUS'),
 ('CHN', 'CHE'),
 ('CHN', 'DEU'),
 ('CHN', 'FRA'),
 ('CHN', 'HKG'),
 ('CHN', 'IDN'),
 ('CHN', 'IND'),
 ('CHN', 'JPN'),
 ('CHN', 'KOR'),
 ('CHN', 'MYS'),
 ('CHN', 'NLD'),
 ('CHN', 'PHL'),
 ('CHN', 'SGP'),
 ('CHN', 'THA'),
 ('CHN', 'USA'),
 ('CHN', 'VNM'),
 ('DEU', 'AUS'),
 ('DEU', 'CHE'),
 ('DEU', 'CHN'),
 ('DEU', 'FRA'),
 ('DEU', 'HKG'),
 ('DEU', 'IDN'),
 ('DEU', 'IND'),
 ('DEU', 'JPN'),
 ('DEU', 'KOR'),
 ('DEU', 'MYS'),
 ('DEU', 'NLD'

: 

### Sector Dictionary
0. Category 1
</br> Description: Agriculture, forestry, fishing, food, beverages, tobacco
</br> HS Goods: 972

1. Category 2
</br>Description: Mining, quarrying, refinery, fuels, chemicals, electricity, water, waste treatment
</br>HS Goods: 983

2. Category 3
</br>Description: Construction, wood, glass, stone, basic metals, housing, electrical appliances, furniture
</br>HS Goods: 1313

3. Category 4
</br>Description: Textile, apparel, shoes, jewelry, leather
</br>HS Goods: 895

4. Category 5
</br>Description: Transport equipment and services, travel, postal services
</br>HS Goods: 180

5. Category 6
</br>Description: ICT, media, computers, business and financial services
</br>HS Goods: 441

6. Category 7
</br>Description: Health, pharmaceuticals, education, cultural, sport
</br>HS Goods: 178

7. Category 8
</br>Description: Government, military and other
</br>HS Goods: 139

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#do normalisation
data_to_normalize = training_data_tensor[:, :, :8]
normalized_data, scaler = normalize_dataset(data_to_normalize, normalizer=args.normaliser,column_wise=True)
remaining_features = training_data_tensor[:, :, 8:]
# Get the shape dimensions
T, N, _ = remaining_features.shape

# Initialize the scaler with the desired feature range (0, 1)
scaler2 = MinMaxScaler(feature_range=(-1, 1))

# Reshape the first column of remaining_features to 2D (T*N, 1)
col_data = remaining_features[:, :, 0].reshape(-1, 1)

# Fit and transform the column data using the scaler
col_scaled = scaler2.fit_transform(col_data)
# Concatenate along the last axis
normalized_training_data = np.concatenate((normalized_data, col_scaled), axis=-1)

# 2. Group data into overlapping windows of 7 time periods (3 input + 3 ahead).
windows = group_into_windows(normalized_training_data, window_size=6)
print("Windows shape (num_samples, input_len, N, D):", windows.shape)

# 3. Split each window into 3 input time periods and a single target (3 steps forward).
x, y = split_input_target_direct(windows, input_len=3, horizon=3)
print("Input shape (num_samples, horizon, N, D):", x.shape)
print("Target shape (num_samples, N, D):", y.shape)

# 4. Perform train/validation split.
x_train, y_train, x_val, y_val = train_val_split(x, y, val_ratio=0.2)
print("Train samples:", x_train.shape[0])
print("Validation samples:", x_val.shape[0])

x_train_tensor=torch.tensor(x_train, dtype=torch.float32)
y_train_tensor=torch.tensor(y_train, dtype=torch.float32)
train_dataset=TensorDataset(x_train_tensor, y_train_tensor)

x_val_tensor=torch.tensor(x_val, dtype=torch.float32)
y_val_tensor=torch.tensor(y_val, dtype=torch.float32)
val_dataset=TensorDataset(x_val_tensor, y_val_tensor)

# Create the dataset and data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

Normalize the dataset by MinMax11 Normalization
Windows shape (num_samples, input_len, N, D): (9, 6, 272, 9)
Input shape (num_samples, horizon, N, D): (9, 3, 272, 9)
Target shape (num_samples, N, D): (9, 3, 272, 8)
Train samples: 7
Validation samples: 2


## Training algorithm

In [13]:
from datetime import datetime
from AGCRN.model.BasicTrainer import Trainer
from agcrn_model import AGCRNFinal
import os

def run():
    model=AGCRNFinal(args)
    model=model.to(args.device)
    
    for p in model.parameters():
        if p.dim() >= 2:
            nn.init.xavier_uniform_(p)
        else:
            # For biases or 1D parameters, just fill with zeros or some small constant
            nn.init.zeros_(p)

    #load dataset here

    #init loss function, optimizer
    loss=torch.nn.MSELoss().to(args.device)
    optimizer=optim.Adam(model.parameters(),lr=args.lr_init,eps=1.0e-8,weight_decay=0.0,amsgrad=False)

    #learning rate decay
    lr_scheduler=None
    if args.lr_decay:
        print('Applying learning rate decay.')
        lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                            milestones=lr_decay_steps,
                                                            gamma=args.lr_decay_rate)

    #config log path
    current_time = datetime.now().strftime('%Y%m%d%H%M%S')
    current_dir = os.getcwd()
    log_dir = os.path.join(current_dir,'logs')
    args.log_dir = log_dir

    #start training
    trainer = Trainer(model, loss, optimizer, train_loader, val_loader, scaler=scaler, #need to get these 
                    args=args, lr_scheduler=lr_scheduler)
    if args.mode == 'train':
        trainer.train()


### Hyperparameters Tuning

We test a few possible hyperparameters based on the paper and do a 80-20 split of the data and get the best model.

Best Model is lr_init=0.004, embed=10, lr_decay=0.1

In [14]:
run()

2025-04-14 19:29: Experiment log path in: c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs
2025-04-14 19:29: Experiment log path in: c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs
2025-04-14 19:29: Experiment log path in: c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs
2025-04-14 19:29: Experiment log path in: c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs
2025-04-14 19:29: Experiment log path in: c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs
2025-04-14 19:29: Train Epoch 1: 0/2 Loss: 0.366759
2025-04-14 19:29: Train Epoch 1: 0/2 Loss: 0.366759
2025-04-14 19:29: Train Epoch 1: 0/2 Loss: 0.366759
2025-04-14 19:29: Train Epoch 1: 0/2 Loss: 0.366759
2025-04-14 19:29: Train Epoch 1: 0/2 Loss: 0.366759
2025-04-14 19:29: **********Train Epoch 1: averaged Loss: 0.348856, tf_ratio: 0.950061


Applying learning rate decay.
Create Log File in:  c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs\run_lr_init_0.0059745480826864684_embed_6_0.17039700074755165.log
output shape: torch.Size([4, 3, 272, 8])
label shape: torch.Size([4, 3, 272, 8])
output shape: torch.Size([3, 3, 272, 8])
label shape: torch.Size([3, 3, 272, 8])


2025-04-14 19:29: **********Train Epoch 1: averaged Loss: 0.348856, tf_ratio: 0.950061
2025-04-14 19:29: **********Train Epoch 1: averaged Loss: 0.348856, tf_ratio: 0.950061
2025-04-14 19:29: **********Train Epoch 1: averaged Loss: 0.348856, tf_ratio: 0.950061
2025-04-14 19:29: **********Train Epoch 1: averaged Loss: 0.348856, tf_ratio: 0.950061
2025-04-14 19:29: **********Val Epoch 1: average Loss: 0.333246
2025-04-14 19:29: **********Val Epoch 1: average Loss: 0.333246
2025-04-14 19:29: **********Val Epoch 1: average Loss: 0.333246
2025-04-14 19:29: **********Val Epoch 1: average Loss: 0.333246
2025-04-14 19:29: **********Val Epoch 1: average Loss: 0.333246
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Train Epoch 2: 0/2 Loss: 0.343841
2025-04-14 19:29: Train Epoch 2: 0/2 Loss: 0.343841
2025-04-

output shape: torch.Size([2, 3, 272, 8])
label shape: torch.Size([2, 3, 272, 8])
last_output shape: torch.Size([2, 272, 8])
output shape: torch.Size([4, 3, 272, 8])
label shape: torch.Size([4, 3, 272, 8])
output shape: torch.Size([3, 3, 272, 8])
label shape: torch.Size([3, 3, 272, 8])


2025-04-14 19:29: **********Train Epoch 2: averaged Loss: 0.345237, tf_ratio: 0.945098
2025-04-14 19:29: **********Train Epoch 2: averaged Loss: 0.345237, tf_ratio: 0.945098
2025-04-14 19:29: **********Train Epoch 2: averaged Loss: 0.345237, tf_ratio: 0.945098
2025-04-14 19:29: **********Train Epoch 2: averaged Loss: 0.345237, tf_ratio: 0.945098
2025-04-14 19:29: **********Val Epoch 2: average Loss: 0.323181
2025-04-14 19:29: **********Val Epoch 2: average Loss: 0.323181
2025-04-14 19:29: **********Val Epoch 2: average Loss: 0.323181
2025-04-14 19:29: **********Val Epoch 2: average Loss: 0.323181
2025-04-14 19:29: **********Val Epoch 2: average Loss: 0.323181
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Train Epoch 3: 0/2 Loss: 0.349874
2025-04-14 19:29: Train Epoch 3: 0/2 Loss: 0.349874
2025-04-

...

output shape: torch.Size([4, 3, 272, 8])
label shape: torch.Size([4, 3, 272, 8])
output shape: torch.Size([3, 3, 272, 8])
label shape: torch.Size([3, 3, 272, 8])
output shape: torch.Size([2, 3, 272, 8])
label shape: torch.Size([2, 3, 272, 8])
last_output shape: torch.Size([2, 272, 8])


2025-04-14 19:29: **********Val Epoch 49: average Loss: 0.255744
2025-04-14 19:29: **********Val Epoch 49: average Loss: 0.255744
2025-04-14 19:29: **********Val Epoch 49: average Loss: 0.255744
2025-04-14 19:29: **********Val Epoch 49: average Loss: 0.255744
2025-04-14 19:29: **********Val Epoch 49: average Loss: 0.255744
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Train Epoch 50: 0/2 Loss: 0.250962
2025-04-14 19:29: Train Epoch 50: 0/2 Loss: 0.250962
2025-04-14 19:29: Train Epoch 50: 0/2 Loss: 0.250962
2025-04-14 19:29: Train Epoch 50: 0/2 Loss: 0.250962
2025-04-14 19:29: Train Epoch 50: 0/2 Loss: 0.250962
2025-04-14 19:29: **********Train Epoch 50: averaged Loss: 0.278291, tf_ratio: 0.124089
2025-04-14 19:29: **********Train Epoch 50: averaged Loss: 0.278291, tf_ratio: 0.124089
2025-04-14 19:

output shape: torch.Size([4, 3, 272, 8])
label shape: torch.Size([4, 3, 272, 8])
output shape: torch.Size([3, 3, 272, 8])
label shape: torch.Size([3, 3, 272, 8])


2025-04-14 19:29: **********Val Epoch 50: average Loss: 0.255729
2025-04-14 19:29: **********Val Epoch 50: average Loss: 0.255729
2025-04-14 19:29: **********Val Epoch 50: average Loss: 0.255729
2025-04-14 19:29: **********Val Epoch 50: average Loss: 0.255729
2025-04-14 19:29: **********Val Epoch 50: average Loss: 0.255729
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Current best model saved!
2025-04-14 19:29: Total training time: 0.1935min, best loss: 0.255729
2025-04-14 19:29: Total training time: 0.1935min, best loss: 0.255729
2025-04-14 19:29: Total training time: 0.1935min, best loss: 0.255729
2025-04-14 19:29: Total training time: 0.1935min, best loss: 0.255729
2025-04-14 19:29: Total training time: 0.1935min, best loss: 0.255729
2025-04-14 19:29: Saving current best model to c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\model\logs/

output shape: torch.Size([2, 3, 272, 8])
label shape: torch.Size([2, 3, 272, 8])
last_output shape: torch.Size([2, 272, 8])


# Model Evaluation

## Data Transformation to pipeline data into model (for 2021 to 2023 % change volumes)

In [16]:
# temp=pd.read_csv('../data/final/final_modeL_data.csv',header=0)
# temp.head()
# temp.drop(columns=['Unnamed: 0'],inplace=True)
# temp=temp[(temp['country_a']!='ARE')& (temp['country_b']!='ARE')]
# temp=temp[(temp['year']>=2020) & (temp['year']<=2023)]
# temp_only_d=temp[['D','country_a','country_b','year']]
# temp.drop(columns=['D'],inplace=True)
# temp.sort_values(by=['country_a','country_b','year'],inplace=True)
# bec_columns = [f'bec_{i}' for i in range(1, 9)]
# for col in bec_columns:
#     # Create a new column to store the percentage change.
#     temp[f'pct_{col}'] = temp.groupby(['country_a', 'country_b'])[col].pct_change() * 100
# temp.dropna(inplace=True)
# temp.reset_index(drop=True,inplace=True)
# temp=temp.merge(temp_only_d, on=['country_a','country_b','year'], how='left')
# temp.drop(columns=bec_columns,inplace=True)
# temp.rename(columns={'pct_bec_1':'bec_1','pct_bec_2':'bec_2','pct_bec_3':'bec_3','pct_bec_4':'bec_4','pct_bec_5':'bec_5','pct_bec_6':'bec_6','pct_bec_7':'bec_7','pct_bec_8':'bec_8'},inplace=True)
# temp.to_csv('../data/final/without_ARE_pct_2021_2023.csv',index=False)

,Unnamed: 0,country_a,country_b,bec_1,bec_2,bec_3,bec_4,bec_5,bec_6,bec_7,bec_8,D,year
0,0,ARE,AUS,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.060739e+09,0.620151,2006
1,1,ARE,CHE,3.797882e+06,1.355991e+06,6.939408e+06,3.759398e+08,2.690339e+06,9.364289e+05,4.422947e+07,9.824469e+07,0.586050,2006
2,2,ARE,CHN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.953953e+09,0.635445,2006
3,3,ARE,DEU,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.159614e+08,0.566027,2006
4,4,ARE,FRA,4.972880e+07,4.680296e+07,9.306163e+07,7.307789e+07,9.781168e+07,1.751175e+07,1.162716e+08,2.462986e+08,0.620551,2006


#### Setting Hyperparameters

In [41]:
args_actual=Args()
args_actual.set_args(embed_dim=10, lr_init=0.005489139587271934,lr_decay_rate=0.18605505546333992, rnn_units=64, num_layers=2, weight_decay=3.0450041080579258e-05) #based on best model by Optuna


### Pipelining the data into the Model

In [31]:
def csv_to_tensor_run(csv_file
                    #   ,sentiment_dict,year_nlp=2023
                      ):
    """
    Reads a CSV file with columns:
      country1, country2, sector1, sector2, ..., sector8, sentiment, year
    and returns a tensor of shape (T, N, D), where:
      T = number of years,
      N = number of unique country pairs,
      D = num of sectors + features.
    Also returns the sorted list of years and country pair nodes.
    """
    def change_sentiment_index(df1,dict,year):
        """
        Change the sentiment index of the dataframe based on NLP model. (For now, it replaces based on year)
        """
        for country_pair, sentiment in dict.items():
            # Extract the country pair from the tuple
            country_a, country_b = country_pair.split('-')
            # Update the sentiment index for the specific year and country pair
            df1.loc[(df1['year'] == year) & (df1['country_a'] == country_a) & (df1['country_b'] == country_b), 'sentiment_index'] = sentiment
            df1.loc[(df1['year'] == year) & (df1['country_a'] == country_b) & (df1['country_b'] == country_a), 'sentiment_index'] = sentiment
        return df1
    # Read the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    # df=change_sentiment_index(df,sentiment_dict,year_nlp)

    # # Transform tradeagreementindex and sentiment_index into D
    # df['D']=1+(-1)*0.5*df['tradeagreementindex']+(-1)*0.5*df['sentiment_index']
    # df=df.drop(columns=['sentiment_index','tradeagreementindex'],axis=1)   

    T= len(df['year'].unique())
    years= sorted(df['year'].unique())
    # Get all unique country pairs
    pairs_df = df[['country_a', 'country_b']].drop_duplicates()
    # Create a sorted list of tuples (country1, country2) for consistent node ordering
    country_pairs = sorted([tuple(x) for x in pairs_df.values])
    N = len(country_pairs)
    
    # Number of features (8 sectors + 1 sentiment)
    D = 9

    # Initialize an empty numpy array for the tensor data
    tensor_data = np.empty((T, N, D), dtype=float)
    
    # Loop over each year and each country pair to fill in the tensor
    for t, year in enumerate(years):
        # Get data for the current year
        df_year = df[df['year'] == year]
        for n, (c1, c2) in enumerate(country_pairs):
            # Filter rows for the current country pair
            row = df_year[(df_year['country_a'] == c1) & (df_year['country_b'] == c2)]
            if not row.empty:
                # Extract the 8 sector columns and the sentiment column.
                # Assumes these columns are named exactly as shown.
                features = row.iloc[0][['bec_1', 'bec_2', 'bec_3', 'bec_4', 
                                         'bec_5', 'bec_6', 'bec_7', 'bec_8', 'D']].values
                tensor_data[t, n, :] = features.astype(float)
            else:
                # If a record is missing for a given year/country pair, fill with zeros (or choose another strategy)
                tensor_data[t, n, :] = np.zeros(D)
                
    return tensor_data, years, country_pairs

def group_into_windows(tensor_data, window_size):
    """
    Given a tensor of shape (T, N, D), group the data into overlapping windows.
    Each window is of length window_size
    Returns a numpy array of shape (num_samples, window_size, N, D).
    """
    T, N, D = tensor_data.shape
    num_samples = T - window_size + 1  # sliding window with stride 1
    windows = []
    for i in range(num_samples):
        window = tensor_data[i: i + window_size]  # shape: (window_size, N, D)
        windows.append(window)
    windows = np.stack(windows)  # shape: (num_samples, window_size, N, D)
    return windows

def split_input_target_direct(windows, input_len, horizon=3):
    """
    Splits each window into input and a single target that is horizon steps forward.
    
    windows: numpy array of shape (num_samples, window_size, N, D)
              where window_size = input_len + horizon.
    input_len: number of time steps used as input.
    horizon: steps forward to pick the target (here, horizon=3).
    
    Returns:
      x: inputs of shape (num_samples, input_len, N, D)
      y: targets of shape (num_samples, N, 8), which are the first 8 features of the target time step.
    """
    # x: first input_len time steps (e.g., years 2006-2009 if input_len=4)
    x = windows[:, :input_len]  
    # y_full: the time step exactly horizon steps forward (i.e., index input_len + horizon - 1)
    # y_full = windows[:, input_len + horizon-1]  
    y_full = windows[:, input_len:input_len + horizon]
    # y: only the first 8 features from the predicted time step (ignoring sentiment_index and tradeagreementindex)
    y = y_full[..., :8]
    return x, y

def train_val_split(x, y, val_ratio=0.2):
    """
    Splits the data into train and validation sets by ratio.
    """
    num_samples = x.shape[0]
    split_index = int(num_samples * (1 - val_ratio))
    x_train, y_train = x[:split_index], y[:split_index]
    x_val, y_val = x[split_index:], y[split_index:]
    return x_train, y_train, x_val, y_val



In [ ]:
from AGCRN.lib.dataloader import normalize_dataset

test_data_tensor, years, country_pairs_model = csv_to_tensor_run('./data/final/without_ARE_pct_2021_2023.csv'
                                                                #  ,sentiment_dict=sentiment_dict,year_nlp=year_nlp
                                                                 )

### Inspect country_pairs to ensure it is the same as initial_nums to perform data transformation later

In [33]:
country_pairs_model

[('AUS', 'CHE'),
 ('AUS', 'CHN'),
 ('AUS', 'DEU'),
 ('AUS', 'FRA'),
 ('AUS', 'HKG'),
 ('AUS', 'IDN'),
 ('AUS', 'IND'),
 ('AUS', 'JPN'),
 ('AUS', 'KOR'),
 ('AUS', 'MYS'),
 ('AUS', 'NLD'),
 ('AUS', 'PHL'),
 ('AUS', 'SGP'),
 ('AUS', 'THA'),
 ('AUS', 'USA'),
 ('AUS', 'VNM'),
 ('CHE', 'AUS'),
 ('CHE', 'CHN'),
 ('CHE', 'DEU'),
 ('CHE', 'FRA'),
 ('CHE', 'HKG'),
 ('CHE', 'IDN'),
 ('CHE', 'IND'),
 ('CHE', 'JPN'),
 ('CHE', 'KOR'),
 ('CHE', 'MYS'),
 ('CHE', 'NLD'),
 ('CHE', 'PHL'),
 ('CHE', 'SGP'),
 ('CHE', 'THA'),
 ('CHE', 'USA'),
 ('CHE', 'VNM'),
 ('CHN', 'AUS'),
 ('CHN', 'CHE'),
 ('CHN', 'DEU'),
 ('CHN', 'FRA'),
 ('CHN', 'HKG'),
 ('CHN', 'IDN'),
 ('CHN', 'IND'),
 ('CHN', 'JPN'),
 ('CHN', 'KOR'),
 ('CHN', 'MYS'),
 ('CHN', 'NLD'),
 ('CHN', 'PHL'),
 ('CHN', 'SGP'),
 ('CHN', 'THA'),
 ('CHN', 'USA'),
 ('CHN', 'VNM'),
 ('DEU', 'AUS'),
 ('DEU', 'CHE'),
 ('DEU', 'CHN'),
 ('DEU', 'FRA'),
 ('DEU', 'HKG'),
 ('DEU', 'IDN'),
 ('DEU', 'IND'),
 ('DEU', 'JPN'),
 ('DEU', 'KOR'),
 ('DEU', 'MYS'),
 ('DEU', 'NLD'

In [34]:
# Inspect data to check
print("Data for year {}:".format(years[0]))
print(test_data_tensor[0])  # prints the data for all nodes/features for the first year


print("Features for {} in {}:".format(country_pairs[0], years[0]))
print(test_data_tensor[0, 0, :]) # prints the features for the first country pair in the first year

Data for year 2021:
[[-23.76676415   5.44830872   8.48811764 ... -16.42896358   0.16023751
    0.80989102]
 [ -4.32133521  28.1916725   27.40833829 ...   4.54033474 -27.55138592
    0.86075767]
 [ 30.04506149  -6.13320462   7.47301508 ... -19.3816128   10.17375255
    0.83792047]
 ...
 [  4.71237695  53.61440096  27.75656542 ...   2.92823602  29.68103916
    0.63215932]
 [-11.86357687  46.59080175  10.18924034 ...   4.79350723  45.04526876
    0.64640775]
 [ 14.98410578  23.31365631  36.93566773 ...  11.27367646  10.96054528
    0.85279134]]
Features for ('AUS', 'CHE') in 2021:
[-23.76676415   5.44830872   8.48811764 -36.85539035 -14.90805586
 -14.90717037 -16.42896358   0.16023751   0.80989102]


In [35]:
from sklearn.preprocessing import MinMaxScaler

#do normalisation
data_to_normalize = test_data_tensor[:, :, :8]
normalized_data, scaler = normalize_dataset(data_to_normalize, normalizer=args.normaliser,column_wise=True)
remaining_features = test_data_tensor[:, :, 8:]

# Get the shape dimensions
T, N, _ = remaining_features.shape

# Initialize the scaler with the desired feature range (-1, 1)
scaler2 = MinMaxScaler(feature_range=(-1, 1))

# Reshape the first column of remaining_features to 2D (T*N, 1)
col_data = remaining_features[:, :, 0].reshape(-1, 1)

# Fit and transform the column data using the scaler
col_scaled = scaler2.fit_transform(col_data)

# Reshape back to the original shape (T, N, 1)
col_scaled = col_scaled.reshape(T, N, 1)
# Concatenate along the last axis
normalized_test_data = np.concatenate((normalized_data, col_scaled), axis=-1)
test_x_tensor=torch.tensor(normalized_test_data, dtype=torch.float32)
test_x_tensor = test_x_tensor.unsqueeze(0)  

Normalize the dataset by MinMax11 Normalization


### Single pass of the model

In [42]:
from agcrn_model import AGCRNFinal

#load model and previously saved states
model=AGCRNFinal(args_actual)
model=model.to(args_actual.device)
model.load_state_dict(torch.load('./logs/best_model_69.pth',map_location=torch.device('cpu')))
model.eval()
with torch.no_grad():
    test_x_tensor = test_x_tensor.to(args_actual.device)
    predictions = model(test_x_tensor,None,0)
    predictions = predictions.cpu().numpy()

#convert back to original scale
predictions = scaler.inverse_transform(predictions[0, :, :, :8])

### Convert % change predictions into absolute values

In [ ]:
initial_nums=pd.read_csv('./data/final/without_ARE_2021_2023.csv',header=0)

# we only need 2023 data to compute forecasted 2024, 2025 and 2026 values
initial_nums=initial_nums[initial_nums['year']==2023]
initial_nums.reset_index(drop=True,inplace=True)

#### Check for ordering of country pairs before appending back to dataframe (should have no outputs)

In [44]:
# Group the dataframe indices by country pairs
for country_pair in country_pairs:
    country_a, country_b = country_pair
    
    # Extract the subset of dataframe where country_a and country_b match
    subset = initial_nums[(initial_nums['country_a'] == country_a) & (initial_nums['country_b'] == country_b)]
    
    #check for any country pair/year missing 
    if subset.empty:
       print(f'country_pair: ({country_a}, {country_b}) is missing')



### applying percentage change to absolute values to get forecasted 24,25,26 values

In [45]:
bec_cols=[f'bec_{i}' for i in range(1, 9)]
future_years = [2024, 2025, 2026]
predicted=[]

for pair_idx, row in initial_nums.iterrows():
    # convert base values to numpy array and float
    base_values = row[bec_cols].values.astype(float)
    
    # copy just for saving the initial values
    current_values = base_values.copy()
    
    # Apply the percentage changes for each future year
    for year_offset, future_year in enumerate(future_years):
        # extract the 8 % changes for country pair in forecast_year
        pct_change = predictions[year_offset, pair_idx, :]
        # convert to factor change
        factor = 1 + pct_change / 100.0
        
        # update current values
        current_values = current_values * factor
        
        # build dictionary to build final dataframe
        row_dict = {
            'country_a': row['country_a'],
            'country_b': row['country_b'],
            'year': future_year
        }
        # add each bec column
        for col_idx, col in enumerate(bec_cols):
            row_dict[col] = current_values[col_idx]
        
        # Add the row to our list
        predicted.append(row_dict)

# Create a new df containing the predicted absolute trade volumes for 2024, 2025, 2026
predictions_df = pd.DataFrame(predicted)


### Pipeline data to frontend

In [ ]:
#Put exports and imports in the same row and transform column names
temp=pd.merge(predictions_df,predictions_df,how='outer',left_on=['country_a','country_b','year'],right_on=['country_b','country_a','year'],suffixes=('_export_A_to_b', '_import_A_from_B'))
temp.drop(columns=['country_a_import_A_from_B','country_b_import_A_from_B'],inplace=True)
temp.rename(columns={'country_a_export_A_to_b':'country_a','country_b_export_A_to_b':'country_b'},inplace=True)
temp['country_pair'] = temp.apply(
    lambda x: '_'.join(sorted([x['country_a'], x['country_b']])), 
    axis=1
)
temp = temp.drop_duplicates(subset=['country_pair', 'year'], keep='first')
temp = temp.drop('country_pair', axis=1)

#add additional columns for frontend
bec_export_cols=[f'bec_{i}_export_A_to_b' for i in range(1, 9)]
bec_import_cols=[f'bec_{i}_import_A_from_B' for i in range(1, 9)]
temp['total_export_A_to_B']=temp[bec_export_cols].sum(axis=1)
temp['total_import_A_from_B']=temp[bec_import_cols].sum(axis=1)
temp['trade_volume']=temp['total_export_A_to_B']+temp['total_import_A_from_B']
temp['scenario']='forecast'

#reorder columns for frontend
temp=temp[['country_a','country_b','year','total_export_A_to_B','total_import_A_from_B','trade_volume']+bec_export_cols+bec_import_cols+['scenario']]

# # #filter only 2026 data and add scenario column for frontend
# temp=temp[temp['year']==2026] #comment out if need 2024 to 2026 data as well

temp['scenario']='postshock'

baseline=pd.read_csv('./data/final/2026_baseline_forecast.csv',header=0)
final=pd.concat([baseline,temp],axis=0,ignore_index=True)
final.reset_index(drop=True,inplace=True)

### NOTE: 
temp is the final dataframe that contains the same columns as sample_2026.csv 

In [ ]:
#Check temp columns
final.columns

Index(['country_a', 'country_b', 'year', 'total_export_A_to_B',
       'total_import_A_from_B', 'trade_volume', 'bec_1_export_A_to_b',
       'bec_2_export_A_to_b', 'bec_3_export_A_to_b', 'bec_4_export_A_to_b',
       'bec_5_export_A_to_b', 'bec_6_export_A_to_b', 'bec_7_export_A_to_b',
       'bec_8_export_A_to_b', 'bec_1_import_A_from_B', 'bec_2_import_A_from_B',
       'bec_3_import_A_from_B', 'bec_4_import_A_from_B',
       'bec_5_import_A_from_B', 'bec_6_import_A_from_B',
       'bec_7_import_A_from_B', 'bec_8_import_A_from_B', 'scenario'],
      dtype='object')